In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/pulmonary.csv")

In [3]:
cvp=pd.read_csv("CVP.csv")

In [4]:
burns_merged = pd.merge(cvp[['patientunitstayid','val']], burns, how='right', on='patientunitstayid')

In [5]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [6]:
burns_merged_2 = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns_merged, how='right', on='patientunitstayid')

In [7]:
burns_merged_2['mods_ratio'] = 0

In [8]:
burns_merged_2

,patientunitstayid,pao2,fio2,val,gender,age,ethnicity,apacheadmissiondx,admissionheight,hospitaladmitoffset,...,Hgb,WBC x 1000,platelets x 1000,RBC,bicarbonate,MCV,MCHC,MCH,RDW,mods_ratio
0,141337,-1.0,-1.0,NaN,Female,72.0,Hispanic,Thoracotomy for other reasons,154.9,-7.000000,...,11.7,10.30,257.0,3.99,26.0,86.5,33.9,29.3,14.2,0
1,141436,141.0,40.0,NaN,Male,46.0,Hispanic,"Arrest, respiratory (without cardiac arrest)",167.6,0.000000,...,15.0,9.70,247.0,4.97,27.0,88.9,33.9,30.2,13.6,0
2,141462,-1.0,-1.0,NaN,Male,80.0,Caucasian,"Sepsis, pulmonary",170.2,-2782.721559,...,12.5,14.50,367.0,4.29,28.0,86.2,33.8,29.1,13.8,0
3,141515,85.0,70.0,6.974684,Female,68.0,Caucasian,"Sepsis, unknown",165.1,-1092.000000,...,10.4,10.00,30.0,3.29,20.0,92.1,34.3,31.6,16.7,0
4,141585,76.0,50.0,NaN,Male,63.0,Caucasian,Emphysema/bronchitis,185.4,-22269.000000,...,8.7,13.80,167.0,2.94,45.0,101.7,29.1,29.6,15.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32826,3353198,52.0,50.0,NaN,Female,66.0,Caucasian,Coma/change in level of consciousness (for hep...,157.5,-68542.000000,...,7.6,11.90,270.0,2.67,34.0,90.0,32.0,28.0,17.5,0
32827,3353200,70.0,30.0,NaN,Female,66.0,Caucasian,"Hypovolemia (including dehydration, Do not inc...",157.5,-133891.000000,...,8.8,12.30,179.0,2.34,28.0,87.0,34.0,29.0,18.5,0
32828,3353201,329.0,100.0,NaN,Female,66.0,Caucasian,"Effusions, pleural",157.5,-58037.000000,...,8.7,10.40,176.0,2.89,38.0,91.0,33.0,30.0,17.3,0
32829,3353226,-1.0,-1.0,NaN,Female,79.0,African American,"Effusions, pleural",121.9,-66.000000,...,6.9,8.40,186.0,2.29,29.0,95.0,32.0,30.0,19.1,0


In [9]:
for i in range(len(burns_merged_2)):
    if (burns_merged_2.iloc[i]['pao2'] != -1) & (burns_merged_2.iloc[i]['fio2'] != -1):
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 300) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 226):
            burns_merged_2['mods_ratio'].iloc[i] = 1
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 225) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 151):
            burns_merged_2['mods_ratio'].iloc[i] = 2
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 150) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 76):
            burns_merged_2['mods_ratio'].iloc[i] = 3
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 0) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 75):
            burns_merged_2['mods_ratio'].iloc[i] = 4

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
def platelets(item):
    temp = 0
    if (item >= 81) & (item <= 120):
        temp += 1
    elif (51 <= item) & (item <= 80):
        temp += 2
    elif (21 <= item) & (item <= 50):
        temp += 3
    elif (20 >= item):
        temp += 4
    return temp

In [11]:
burns_merged_2['mods_platelets'] = burns_merged_2['platelets x 1000'].apply(platelets)

In [12]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [13]:
burns_merged_3 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged_2, how='right', on='patientunitstayid')

In [14]:
def bilirubin(item):
    temp = 0
    if (item >= 1.3) & (item <= 3.5):
        temp += 1
    elif (3.6 <= item) & (item <= 7.0):
        temp += 2
    elif (7.0 <= item) & (item <= 14.0):
        temp += 3
    elif (14.0 <= item):
        temp += 4
    return temp

In [15]:
burns_merged_3['mods_bilirubin'] = burns_merged_3['bilirubin'].apply(bilirubin)

In [16]:
burns_merged_3['val_fill'] = burns_merged_3['val'].fillna(burns_merged_3['val'].mean())

In [17]:
burns_merged_3['PAHR'] = (burns_merged_3['heartrate'] * burns_merged_3['val'])/burns_merged_3['meanbp']

In [18]:
burns_merged_3['PAHRF'] = (burns_merged_3['heartrate'] * burns_merged_3['val_fill'])/burns_merged_3['meanbp']

In [19]:
burns_merged_3['PAHRF'].mean()

24.492081526419156

In [20]:
def ad_pressure(item):
    temp = 0
    if (item >= 10.1) & (item <= 15):
        temp += 1
    elif (item >= 15.1) & (item <= 20):
        temp += 2
    elif (item >= 20.1) & (item <= 30):
        temp += 3
    elif (item >= 30.1):
        temp += 4
    return temp

In [21]:
burns_merged_3['mods_pressure'] = burns_merged_3['PAHR'].apply(ad_pressure)

In [22]:
pd.set_option('display.max_rows', 1000)

In [23]:
burns_merged_3.isna().sum()

patientunitstayid          0
bilirubin                  0
pao2                       0
fio2                       0
val                    29531
gender                     0
age                        0
ethnicity                  0
apacheadmissiondx          0
admissionheight            0
hospitaladmitoffset        0
unittype                   0
unitadmitsource            0
unitstaytype               0
admissionweight            0
intubated                  0
dialysis                   0
temperature                0
respiratoryrate            0
heartrate                  0
meanbp                     0
hematocrit                 0
actualicumortality         0
preopmi                    0
preopcardiaccath           0
ptcawithin24h              0
meds                       0
verbal                     0
motor                      0
eyes                       0
thrombolytics              0
aids                       0
hepaticfailure             0
lymphoma                   0
immunosuppress

In [24]:
burns_merged_3['GCS'] = burns_merged_3['eyes'] + burns_merged_3['motor'] + burns_merged_3['verbal']

In [25]:
def GCS(item):
    temp = 0
    if (item >= 13) & (item <= 14):
        temp += 1
    elif (10 <= item) & (item <= 12):
        temp += 2
    elif (6 <= item) & (item <= 9):
        temp += 3
    elif (6 > item):
        temp += 4
    return temp

In [26]:
burns_merged_3['mods_GCS'] = burns_merged_3['GCS'].apply(GCS)

In [27]:
def creatinine(item):
    temp = 0
    if (item >= 1.2) & (item <= 2.2):
        temp += 1
    elif (2.3 <= item) & (item <= 3.9):
        temp += 2
    elif (4 <= item) & (item <= 5.7):
        temp += 3
    elif (5.7 < item):
        temp += 4
    return temp

In [28]:
burns_merged_3['mods_creatinine'] = burns_merged_3['creatinine'].apply(creatinine)

In [29]:
burns_merged_3.columns

Index(['patientunitstayid', 'bilirubin', 'pao2', 'fio2', 'val', 'gender',
       'age', 'ethnicity', 'apacheadmissiondx', 'admissionheight',
       'hospitaladmitoffset', 'unittype', 'unitadmitsource', 'unitstaytype',
       'admissionweight', 'intubated', 'dialysis', 'temperature',
       'respiratoryrate', 'heartrate', 'meanbp', 'hematocrit',
       'actualicumortality', 'preopmi', 'preopcardiaccath', 'ptcawithin24h',
       'meds', 'verbal', 'motor', 'eyes', 'thrombolytics', 'aids',
       'hepaticfailure', 'lymphoma', 'immunosuppression', 'cirrhosis',
       'activetx', 'ima', 'midur', 'ventday1', 'oobventday1', 'oobintubday1',
       'diabetes', 'diagnosisstring', 'potassium', 'creatinine', 'sodium',
       'BUN', 'glucose', 'chloride', 'calcium', 'Hgb', 'WBC x 1000',
       'platelets x 1000', 'RBC', 'bicarbonate', 'MCV', 'MCHC', 'MCH', 'RDW',
       'mods_ratio', 'mods_platelets', 'mods_bilirubin', 'val_fill', 'PAHR',
       'PAHRF', 'mods_pressure', 'GCS', 'mods_GCS', 'mods_cre

In [30]:
burns_merged_3['mods_score'] = burns_merged_3[['mods_ratio', 'mods_platelets', 'mods_bilirubin',
        'mods_pressure', 'mods_GCS', 'mods_creatinine']].sum(axis=1)

In [31]:
burns_merged_3['actualicumortality_label'] = burns_merged_3['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [32]:
def mods_predict(item):
    temp = 0
    if (item >= 1) & (item <= 4):
        temp = 1.5
    elif (item >= 5) & (item <= 8):
        temp = 4.0
    elif (item >= 9) & (item <= 12):
        temp = 25
    elif (item >= 13) & (item <= 16):
        temp = 50
    elif (item >= 17) & (item <= 20):
        temp = 75
    elif (item >= 21) & (item <= 24):
        temp = 100
    return temp

In [33]:
burns_merged_3['mods_score'].max()

20

In [34]:
burns_merged_3['mods_prob'] = burns_merged_3['mods_score'].apply(mods_predict)

In [35]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_3['actualicumortality_label'].values, burns_merged_3['mods_prob'].values, pos_label=1)

In [36]:
metrics.auc(fpr, tpr)

0.7273617249551576

In [37]:
burns_merged_3['mods_prob']

0         0.0
1         1.5
2         1.5
3        25.0
4         1.5
         ... 
32826     4.0
32827     1.5
32828     1.5
32829     1.5
32830     1.5
Name: mods_prob, Length: 32831, dtype: float64